**Notes before running**:
- Allow Internet access and set GPU runtime.
- If your repo is on GitHub, set `REPO_URL` below; otherwise upload the repo to Drive and set `DRIVE_REPO_PATH`.
- This notebook provides two inference options: `transformers+bitsandbytes` (recommended) and `vLLM` (optional, heavier).

In [1]:
# 1) Install core dependencies (may take a few minutes)
# Adjust versions if your project requires specific ones.
# Adjust versions if your project requires specific ones.
import os
print('Installing packages...')
# Install main libs; vLLM is optional (comment/uncomment as needed)
!pip -q install --upgrade pip
!pip -q install transformers accelerate bitsandbytes safetensors sentencepiece huggingface_hub datasets wandb pandas scipy
# If you want vLLM on Colab (optional, can be heavy): uncomment next line
# !pip -q install vllm
print('Done')

Installing packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.8 MB/s eta 0:00:00
Done


In [2]:
# 2) Mount Google Drive (optional but recommended to persist caches & results)
from google.colab import drive
drive.mount('/content/drive')
# Create directories for HF cache and model offload on Drive to avoid re-downloads
import os
HF_DIR = '/content/drive/MyDrive/hf_cache'
OFFLOAD_DIR = '/content/drive/MyDrive/hf_offload'
os.makedirs(HF_DIR, exist_ok=True)
os.makedirs(OFFLOAD_DIR, exist_ok=True)
os.environ['HF_HOME'] = HF_DIR
os.environ['TRANSFORMERS_CACHE'] = HF_DIR
os.environ['XDG_CACHE_HOME'] = HF_DIR
os.environ['HF_DATASETS_CACHE'] = HF_DIR
os.environ['ACCELERATE_STATE_DIR'] = OFFLOAD_DIR
print('HF cache ->', HF_DIR)

Mounted at /content/drive
HF cache -> /content/drive/MyDrive/hf_cache


In [3]:
# 2.5) Set a Drive output directory for results and checkpoints
import os
DRIVE_OUTPUT_DIR = '/content/drive/MyDrive/subliminal_results'
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)
os.environ['DRIVE_OUTPUT_DIR'] = DRIVE_OUTPUT_DIR
print('DRIVE_OUTPUT_DIR ->', DRIVE_OUTPUT_DIR)
# Optionally set HF cache/offload to Drive (already set above)

DRIVE_OUTPUT_DIR -> /content/drive/MyDrive/subliminal_results


In [4]:
# 3) Clone or copy your project into /content/SubliminalLearning
# Option A: clone from GitHub (set REPO_URL)
REPO_URL = 'https://github.com/GabraxVolz1/SubliminalLearning.git'  # e.g. 'https://github.com/yourname/SubliminalLearning.git'
DRIVE_REPO_PATH = '/content/drive/MyDrive/SubliminalLearning'  # if you uploaded repo to Drive
import os, shutil
TARGET = '/content/SubliminalLearning'
if os.path.exists(TARGET):
    print('Project already present at', TARGET)
elif REPO_URL:
    print('Cloning from', REPO_URL)
    !git clone --depth 1 {REPO_URL} {TARGET}
elif os.path.exists(DRIVE_REPO_PATH):
    print('Copying from Drive', DRIVE_REPO_PATH)
    shutil.copytree(DRIVE_REPO_PATH, TARGET)
else:
    print('No REPO_URL and DRIVE_REPO_PATH not found. Please set one and rerun this cell.')
print('Project directory:', os.path.exists(TARGET) and TARGET or 'NOT FOUND')

Cloning from https://github.com/GabraxVolz1/SubliminalLearning.git
Cloning into '/content/SubliminalLearning'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 87 (delta 18), reused 71 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (87/87), 413.60 KiB | 9.19 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Project directory: /content/SubliminalLearning


In [6]:
# 4) Install project (editable) and any extra requirements if available
import os
PROJECT_DIR = '/content/SubliminalLearning'
if os.path.exists(PROJECT_DIR):
    %cd /content/SubliminalLearning
    # Prefer requirements.txt or pyproject-based install if present
    if os.path.exists('requirements.txt'):
        print('Installing requirements.txt')
        !pip -q install -r requirements.txt
    elif os.path.exists('pyproject.toml'):
        print('Installing project in editable mode (pyproject.toml found)')
        !pip -q install -e .
    else:
        print('No requirements.txt or pyproject.toml found — ensure dependencies are installed manually')
    %cd /content
else:
    print('Project dir not found; cannot install project')

/content/SubliminalLearning
Installing project in editable mode (pyproject.toml found)
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for sl (pyproject.toml) ... done
/content


In [7]:
# 5) Set HF token (so private models can be downloaded)
from getpass import getpass
token = getpass('Enter your Hugging Face token (leave blank to skip): ')
if token:
    import os
    os.environ['HUGGINGFACE_HUB_TOKEN'] = token
    !huggingface-cli login --token {token}  # optional; helps with rate limits
else:
    print('Skipping token set; public models only.')

Enter your Hugging Face token (leave blank to skip): ··········
Skipping token set; public models only.


In [8]:
# 6) Check GPU and memory (important before loading models)
import torch, subprocess
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print('Device:', torch.cuda.get_device_name(0))
    except Exception as e:
        print('Device name error:', e)
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

Torch version: 2.9.0+cu126
CUDA available: True
Device: Tesla T4
name, memory.total [MiB], memory.free [MiB]
Tesla T4, 15360 MiB, 15092 MiB


## Inference options
Choose one of the cells below depending on model runtime preference and GPU capacity. If you have A100 (>=40GB) try `vLLM` or large models; otherwise use `transformers+bitsandbytes` 8-bit for 7B models.

In [10]:
pip install transformers -U

  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sl 0.1.0 requires tokenizers==0.21.1, but you have tokenizers 0.22.1 which is incompatible.


In [11]:
# 7A) Recommended: Load a 7B model with transformers + bitsandbytes (fast to try)
# This attempts an 8-bit load using device_map='auto' and requires bitsandbytes installed.
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.1'
try:
    print('Loading tokenizer...')
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    print('Loading model in 8-bit (this may take a minute)...')
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map='auto', load_in_8bit=True, trust_remote_code=True)
    gen = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map='auto')
    print(gen('Write a concise greeting.', max_new_tokens=30)[0]['generated_text'])
except Exception as e:
    print('Error loading 8-bit model:')
    import traceback
    traceback.print_exc()

Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a concise greeting.
- Hello there! How can I assist you today?


In [ ]:
# 7B) Optional: Start vLLM server (only if you installed vllm and have enough GPU memory)
# Uncomment and run if you want to try vLLM serving.
# Note: vLLM can require a lot of GPU RAM; prefer A100.
# !vllm serve mistralai/Mistral-7B-Instruct-v0.1 --tokenizer-mode mistral &
print('If you installed vLLM and have sufficient GPU memory, run the vLLM serve command in a terminal cell (commented).')

In [ ]:
# 8) Run a small end-to-end pilot of the project (adjust args for your experiment)
# This runs your repo's scripts with small sizes so you can verify the pipeline works on Colab.
import os, subprocess
PROJECT_DIR = '/content/SubliminalLearning'
if not os.path.exists(PROJECT_DIR):
    print('Project not available at', PROJECT_DIR)
else:
    %cd /content/SubliminalLearning
    # Example: generate a small teacher set (10) and checkpoint to Drive
    try:
        print('Generating teacher conversations (n=10)...')
        !python scripts/generate_teacher_conversations.py --count 10 --turns 1 --out {DRIVE_OUTPUT_DIR}/teachers_sample.jsonl
    except Exception as e:
        print('Teacher generation failed:', e)
    # Example: run a tiny student roleplay (limit 5) and save outputs to Drive
    try:
        print('Running student roleplay (pilot)...')
        !python scripts/run_student_roleplay.py --in {DRIVE_OUTPUT_DIR}/teachers_sample.jsonl --out {DRIVE_OUTPUT_DIR}/student_pilot.jsonl --limit 5 --batch-size 2
    except Exception as e:
        print('Student roleplay failed:', e)
    # Example: run ablation driver with tiny settings and save to Drive (uses DRIVE_OUTPUT_DIR by default)
    try:
        print('Running ablation driver (pilot)...')
        !python scripts/ablation_driver.py --teacher {DRIVE_OUTPUT_DIR}/teachers_sample.jsonl --limit 10 --out-dir {DRIVE_OUTPUT_DIR}/ablation_pilot
    except Exception as e:
        print('Ablation driver failed:', e)
    %cd /content

### **Warning: Running the following cells will permanently delete data from your Google Drive. Proceed with caution.**

In [14]:
import os

HF_DIR = '/content/drive/MyDrive/hf_cache'
OFFLOAD_DIR = '/content/drive/MyDrive/hf_offload'
DRIVE_OUTPUT_DIR = '/content/drive/MyDrive/subliminal_results'
DRIVE_REPO_PATH = '/content/drive/MyDrive/SubliminalLearning' # Only if you copied your repo to Drive

print(f"Attempting to remove: {HF_DIR}")
if os.path.exists(HF_DIR):
    !rm -rf '{HF_DIR}'
    print(f"Removed: {HF_DIR}")
else:
    print(f"Directory not found: {HF_DIR}")

print(f"\nAttempting to remove: {OFFLOAD_DIR}")
if os.path.exists(OFFLOAD_DIR):
    !rm -rf '{OFFLOAD_DIR}'
    print(f"Removed: {OFFLOAD_DIR}")
else:
    print(f"Directory not found: {OFFLOAD_DIR}")

print(f"\nAttempting to remove: {DRIVE_OUTPUT_DIR}")
if os.path.exists(DRIVE_OUTPUT_DIR):
    !rm -rf '{DRIVE_OUTPUT_DIR}'
    print(f"Removed: {DRIVE_OUTPUT_DIR}")
else:
    print(f"Directory not found: {DRIVE_OUTPUT_DIR}")

# This path is usually created if you manually uploaded the repo to Drive.
# Based on the notebook history, it was cloned, so this might not exist on your Drive.
print(f"\nAttempting to remove: {DRIVE_REPO_PATH}")
if os.path.exists(DRIVE_REPO_PATH):
    !rm -rf '{DRIVE_REPO_PATH}'
    print(f"Removed: {DRIVE_REPO_PATH}")
else:
    print(f"Directory not found: {DRIVE_REPO_PATH}")

print('\nCleanup complete. Please verify your Google Drive manually.')

Attempting to remove: /content/drive/MyDrive/hf_cache
Directory not found: /content/drive/MyDrive/hf_cache

Attempting to remove: /content/drive/MyDrive/hf_offload
Directory not found: /content/drive/MyDrive/hf_offload

Attempting to remove: /content/drive/MyDrive/subliminal_results
Directory not found: /content/drive/MyDrive/subliminal_results

Attempting to remove: /content/drive/MyDrive/SubliminalLearning
Directory not found: /content/drive/MyDrive/SubliminalLearning

Cleanup complete. Please verify your Google Drive manually.


## Scaling to full experiment
- For large runs (e.g., 400 conversations) use Colab Pro+ A100 or a rented VM with a large GPU to avoid OOM and session time limits.
- Save intermediate outputs to Drive (`/content/drive/MyDrive/...`) to avoid losing progress on disconnects.
- Use `--num`/`--n_students`/`--n_repeats` parameters in your scripts to scale; monitor GPU memory with `!nvidia-smi`.